<a href="https://colab.research.google.com/github/seq2193/so-vits-svc-colab/blob/main/sovits3_0%E4%B8%80%E9%94%AE%E8%AE%AD%E7%BB%83_%E6%8E%A8%E7%90%86%E8%84%9A%E6%9C%AC%EF%BC%8832k_48k%E5%88%86%E6%94%AF%E5%88%87%E6%8D%A2%EF%BC%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

初始项目来源：https://github.com/innnky/so-vits-svc

修改日期：2023年3月13日

原始Colab 链接<a target="_blank" href="https://colab.research.google.com/drive/1_-gh9i-wCPNlRZw6pYF-9UufetcVrGBX?usp=sharing">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [ ]:
# 查看显卡
!nvidia-smi

In [ ]:
#@title 克隆的github仓库
#@markdown ##选择要克隆的github仓库分支

Clone = "32k" #@param ["32k","48k"]
if Clone == "32k":
  !git clone https://github.com/svc-develop-team/so-vits-svc -b 3.0-32k
elif Clone == "48k":
  !git clone https://github.com/svc-develop-team/so-vits-svc -b 3.0-48k

In [ ]:
#@title 安装依赖
%cd /content/so-vits-svc
!pip install pyworld praat-parselmouth scikit-maad gradio
!pip install --upgrade gdown

In [ ]:
#@title 下载必要模型文件
!wget -P hubert/ https://github.com/bshall/hubert/releases/download/v0.1/hubert-soft-0d54a1f4.pt

# 数据集预处理

In [ ]:
#@title 加载Google云端硬盘
#@markdown 加载Google云端硬盘
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


该处理脚本可以一次性预处理多个说话人，并且生成多说话人filelists以及对应的配置文件

只需要将你的数据集按照以下文件结构放到dataset_raw目录下


```
dataset_raw
├───speaker0
│   ├───xxx1-xxx1.wav
│   ├───...
│   └───Lxx-0xx8.wav
└───speaker1
    ├───xx2-0xxx2.wav
    ├───...
    └───xxx7-xxx007.wav

```




In [ ]:
#@title 从谷歌云盘加载打包好的数据集进行预处理
#@markdown **sovits3.0的数据集不再需要特定的文件结构，将数据集的所有wav文件放在同一个文件夹下压缩为zip后上传至谷歌云盘即可，该处理脚本可以一次性预处理多个数据集，处理多个数据集时请依次解压每一个数据集**

#@markdown 数据集名称（**人物的英文/拼音名**，与建数据文件夹时统一；不带zip。）
DATASETNAME = "test"  #@param {type:"string"}
#@markdown 压缩包路径（谷歌盘路径，传到dataset的就不改这个，没有dataset文件夹就新建一个）
ZIP_PATH = "/content/drive/MyDrive/dataset/"  #@param {type:"string"}
ZIP_NAME = ZIP_PATH + DATASETNAME

!unzip -d /content/so-vits-svc/dataset_raw {ZIP_NAME}.zip

In [ ]:
#@title 重采样到32k/48k，根据你选择克隆的github仓库分支自动确定
!python resample.py

./dataset_raw/test
53it [00:11,  4.56it/s]


In [ ]:
#@title 划分训练集 生成配置文件
!python preprocess_flist_config.py

In [ ]:
#@title 生成hubert和f0
!python preprocess_hubert_f0.py

In [ ]:
#@title 至此 数据集预处理制作完毕，将数据集和相关文件保存到谷歌云盘的dataset文件夹下，方便下次训练使用
#压缩dataset文件夹
!zip -r dataset.zip /content/so-vits-svc/dataset
#@markdown 自定义备份到谷歌云盘的dataset文件夹下的数据集文件夹名，避免混淆
dataset_name_drive = "32ktest_dataset"  #@param {type:"string"}
DATASET_PATH_DRIVE = "/content/drive/MyDrive/dataset/" + dataset_name_drive
!mkdir -p {DATASET_PATH_DRIVE}

!cp /content/so-vits-svc/dataset.zip "{DATASET_PATH_DRIVE}"
!cp configs/config.json "{DATASET_PATH_DRIVE}"
!cp filelists/train.txt "{DATASET_PATH_DRIVE}"
!cp filelists/val.txt "{DATASET_PATH_DRIVE}"

In [ ]:
#@title 已经预处理过数据集的话，就可以跳过预处理部分 直接从云盘解压处理过的数据 以及配置文件
#@markdown 从谷歌云盘加载预处理过的数据集，文件夹名和你备份的时候输入的一样
back_up_name = "three_dataset"  #@param {type:"string"}
BACK_UP_DATASET_PATH = "/content/drive/MyDrive/dataset/" + back_up_name
!unzip {BACK_UP_DATASET_PATH}/dataset.zip -d /
!cp {BACK_UP_DATASET_PATH}/config.json /content/so-vits-svc/configs/config.json 
!cp {BACK_UP_DATASET_PATH}/val.txt filelists/val.txt
!cp {BACK_UP_DATASET_PATH}/train.txt filelists/train.txt


# 拷贝云盘上保存的记录点
# !cp /content/drive/MyDrive/G_800.pth logs/48k/
# !cp /content/drive/MyDrive/D_800.pth logs/48k/

# 训练

In [ ]:
#@title  选择是否将训练后的模型保存到谷歌云盘 和 是否使用预模型
#@markdown ##选择你克隆的github仓库的分支和你最开始选的保持一致
Clone = "32k" #@param ["32k","48k"]

#@markdown **将训练后的模型文件保存到谷歌云盘，勾选后，恢复训练时也需要勾选并执行**
Save_to_drive = True #@param {type:"boolean"}
if Save_to_drive:
  !rm -rf /content/so-vits-svc/logs/"{Clone}"
  !mkdir -p /content/drive/MyDrive/"{Clone}"
  !ln -s /content/drive/MyDrive/"{Clone}" /content/so-vits-svc/logs/"{Clone}"

#@markdown **首次训练下载预模型 之后继续训练则使用自己保存的记录点，无需再下载**

#@markdown **使用预模型，下面打钩自动下载并启用**
pre_pth = True #@param {type:"boolean"}
if pre_pth:
  #!wget -P logs/"{Clone}"/ https://huggingface.co/innnky/sovits_pretrained/resolve/main/G_0.pth
  #!wget -P logs/"{Clone}"/ https://huggingface.co/innnky/sovits_pretrained/resolve/main/D_0.pth

  !wget -P logs/"{Clone}"/ https://huggingface.co/datasets/seq2193/sovits_pretrained/resolve/main/so-vits-svc-3.0/G_0.pth
  !wget -P logs/"{Clone}"/ https://huggingface.co/datasets/seq2193/sovits_pretrained/resolve/main/so-vits-svc-3.0/D_0.pth


In [ ]:
#@title  开始训练
#@markdown ##选择你克隆的github仓库的分支和你最开始选的保持一致
Clone = "32k" #@param ["32k","48k"]

#@markdown **开始训练**

#@markdown **启用tensorboard可视化数据**
tensorboard_on = True #@param {type:"boolean"}
if tensorboard_on:
  %load_ext tensorboard
  %tensorboard --logdir logs/"{Clone}"

!python train.py -c configs/config.json -m "{Clone}"


In [ ]:
#@title 手动将训练后的模型文件备份到谷歌云盘
#@markdown 需要自己查看/content/so-vits-svc/logs/48k/文件夹下模型的文件名，手动修改下方命令末尾的文件名
!mv /content/so-vits-svc/logs/48k/G_1000.pth /content/drive/MyDrive
!mv /content/so-vits-svc/logs/48k/D_1000.pth /content/drive/MyDrive

## 加载歌手模型

In [ ]:
#@title 从huggingface下载模型
#@markdown ##选择你克隆的github仓库的分支和你最开始训练的保持一致
Clone = "32k" #@param ["32k","48k"]
pth_link="https://huggingface.co/xxxx/so-vits-svc-acg-models/resolve/main/32k-tsubaki/G_20000.pth" #@param{type:"string"}
config_link="https://huggingface.co/xxxx/so-vits-svc-acg-models/resolve/main/32k-tsubaki/config.json" #@param{type:"string"}
!wget -P /content/so-vits-svc/logs/{Clone} {pth_link}
!wget -O /content/so-vits-svc/configs/config.json  {config_link}

# 推理

In [ ]:
#@markdown 下载示例音频合成 風になる
!gdown --id 1d-npX0KOYW5AE2fh0z1mkX3nFYvyrRii -O /content/so-vits-svc/raw/風になる.wav

In [ ]:
#@title 上传.wav音频文件 
import os
from google.colab import files

save_dir = "/content/so-vits-svc/raw"
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

uploaded = files.upload()
file_name = list(uploaded.keys())[0]
file_path = os.path.join(save_dir, file_name)
with open(file_path, "wb") as f:
    f.write(uploaded[file_name])
print(f"File saved at: {file_path}")

In [ ]:
#手动修改 inference_main.py 推理
from google.colab import files
files.view('inference_main.py')

In [ ]:
#@title 一键推理 inference_main.py 配置文件并合成
#@markdown 17行更改model_path为你自己训练的最新模型记录点 例如：NyaruTaffy.pth
model_path = "logs/32k/G_31000.pth" #@param {type:"string"}
#@markdown 23行clean_names 写待转换的音频名称，不含扩展名
clean_names = "\u98A8\u306B\u306A\u308B" #@param {type:"string"}
#@markdown 24行trans 填写变调半音数量 
trans = "0" #@param {type:"string"}
#@markdown 25行spk_list 填写合成的说话人名称 例如：猫雷 nyaru，塔菲 taffy
spk_list = "teto" #@param {type:"string"}
#@markdown 26行默认-40，嘈杂的音频可以-30，干声保留呼吸可以-50
slice_db = "-40" #@param {type:"string"}
#@markdown 27行音频输出格式:默认flac 或 wav
wav_format = "flac" #@param {type:"string"}


!sed -i -r 's|(model_path =)(\s+)(.+)|\1\2"{model_path}"|g' /content/so-vits-svc/inference_main.py
!sed -i -r 's|(clean_names =)(\s+)(.+)|\1\2["{clean_names}"]|g' /content/so-vits-svc/inference_main.py
!sed -i -r 's|(trans =)(\s+)(.+)|\1\2[{trans}]|g' /content/so-vits-svc/inference_main.py
!sed -i -r 's|(spk_list =)(\s+)(.+)|\1\2["{spk_list}"]|g' /content/so-vits-svc/inference_main.py
!sed -i -r 's|(slice_db =)(\s+)(.+)|\1\2{slice_db}|g' /content/so-vits-svc/inference_main.py
!sed -i -r 's|(wav_format =)(\s+)(.+)|\1\2"{wav_format}"|g' /content/so-vits-svc/inference_main.py
#启动推理
!python /content/so-vits-svc/inference_main.py

In [ ]:
#@title raw目录音频切片器（遇到CUDA错误时需要切片再推理）
!pip install pydub
import os
from pydub import AudioSegment

# 定义输入和输出目录
input_dir = "/content/diff-svc/raw"
output_dir = "/content/diff-svc/raw"

# 遍历输入目录中的所有音频文件
for file_name in os.listdir(input_dir):
    # 仅处理扩展名为 wav 的文件
    if file_name.endswith(".wav"):
        # 加载音频文件
        audio = AudioSegment.from_file(os.path.join(input_dir, file_name), format="wav")
        
        # 计算每60秒的毫秒数
        segment_length_ms = 60 * 1000
        
        # 切割音频文件
        segments = []
        for start_time in range(0, len(audio), segment_length_ms):
            end_time = start_time + segment_length_ms
            segment = audio[start_time:end_time]
            segments.append(segment)
        
        # 保存切割后的音频文件到输出目录
        for i, segment in enumerate(segments):
            output_file_name = f"{os.path.splitext(file_name)[0]}_{i}.wav"
            output_path = os.path.join(output_dir, output_file_name)
            segment.export(output_path, format="wav")


In [ ]:
#@title 精简模型 
#@markdown ##选择你克隆的github仓库的分支和你最开始选的保持一致
Clone = "32k" #@param ["32k","48k"]

import torch ,os
logdir = "/content/so-vits-svc/logs/"
logdir = (f"{logdir}/{Clone}")

new_pth_dir = os.path.join(logdir)

#@markdown 模型名，数字是最新的模型steps

new_model_name = "G_20000.pth"  # @param {type:"string"}
checkpoint_dict = torch.load(f"{new_pth_dir}/{new_model_name}")
iteration = checkpoint_dict['iteration']
learning_rate = checkpoint_dict['learning_rate']
optimizer = checkpoint_dict['optimizer']
saved_state_dict = checkpoint_dict['model']
print(iteration)
#@markdown 输出xxx_epoch.pth；此模型**仅供合成使用**，去除训练信息、体积约为1/3，**无法训练使用**
torch.save({'model': saved_state_dict,
  'iteration': None,
  'optimizer': None,
  'learning_rate': None}, f'{new_pth_dir}/{iteration}_epochs.pth')